In [24]:
import pandas as pd
import numpy as np
from random import shuffle
from osgeo import ogr, osr
from sentinelhub import WmsRequest, WcsRequest, MimeType, CRS, BBox, constants
import logging
from collections import Counter
import datetime
import os
import yaml
from sentinelhub import DataSource
import scipy.sparse as sparse
import scipy
from scipy.sparse.linalg import splu
from skimage.transform import resize
from sentinelhub import CustomUrlParam
from time import time as timer
from time import sleep as sleep
import multiprocessing
import math
import reverse_geocoder as rg
import pycountry
import pycountry_convert as pc
import hickle as hkl
from shapely.geometry import Point, Polygon
import geopandas
from tqdm import tnrange, tqdm_notebook
import math
import boto3
from pyproj import Proj, transform
from timeit import default_timer as timer
from typing import Tuple, List

In [25]:
with open("../config.yaml", 'r') as stream:
    key = (yaml.safe_load(stream))
    API_KEY = key['key']
    AWSKEY = key['awskey']
    AWSSECRET = key['awssecret']

In [33]:
%run ../src/preprocessing/slope.py
%run ../src/preprocessing/indices.py
%run ../src/downloading/utils.py
%run ../src/preprocessing/cloud_removal.py
%run ../src/preprocessing/whittaker_smoother.py
%run ../src/dsen2/utils/DSen2Net.py

# Parameters

In [4]:
year = 2018
dates = ('{}-12-01'.format(str(year - 1)) , '{}-02-01'.format(str(year + 1)))
dates_sentinel_1 = ('{}-01-01'.format(str(year)) , '{}-12-31'.format(str(year)))
SIZE = 9*5
IMSIZE = (7*2) + (SIZE * 14)+2

days_per_month = [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30]
starting_days = np.cumsum(days_per_month)

In [5]:
landscapes = {
    'ethiopia-tigray': (13.540810, 38.177220),
    'kenya-makueni-2': (-1.817109, 37.44563),
    'ghana': (9.259359, -0.83375),
    'niger-koure': (13.18158, 2.478),
    'cameroon-farnorth': (10.596, 14.2722),
    'mexico-campeche': (18.232495, -92.1234215),
    'malawi-rumphi-old': (-11.044, 33.818),
    'malawi-rumphi': (-11.15, 33.246),
    'ghana-sisala-east': (10.385, -1.765),
    'ghana-west-mamprusi': (10.390084, -0.846330),
    'ghana-kwahu': (6.518909, -0.826008),
    'senegal-16b': (15.82585, -15.34166),
    'india-kochi': (9.909, 76.254),
    'india-sidhi': (24.0705, 81.607),
    'brazil-esperito-santo': (-20.147, -40.837),
    'brazil-paraiba': (-22.559943, -44.186629),
    'brazil-goias': (-14.905595, -48.907399),
    'colombia-talima': (4.179529, -74.889171),
    'drc-kafubu': (-11.749636, 27.586622),
    'thailand-khon-kaen': (15.709725, 102.546518),
    'indonesia-west-java': (-6.721101, 108.280949),
    'madagascar': (-18.960152, 47.469587),
    'tanzania': (-6.272258, 36.679824),
    'chile': (-36.431237, -71.872030),
    'indonesia-jakarta': (-6.352580, 106.677072),
    'caf-baboua': (5.765917, 14.791618),   
    'honduras': (14.096664, -88.720304),
    'nicaragua': (12.398014, -86.963042),
    'china': (26.673679, 107.464231),
    'australia-west': (-32.666762, 117.411197),
    'mexico-sonora': (29.244288, -111.243230),
    'south-africa': (-30.981698, 28.727301),
    'maldonado-uraguay': (-34.629250, -55.004331),
    'dominican-rep-la-salvia': (18.872589, -70.462961),
    'guatemala-coban': (15.3, -90.8),
    'senegal-tucker-a': (15.350595, -15.459789),
    'elsalvador-imposible': (13.727334, -90.015579),
    'peru-shatoja-district': (-6.566366, -76.759752),
    'angola-galanga': (-12.104782, 15.151222),
    'morocco-chefchaouen': (34.942560, -4.772589),
    'georgia-imereti': (42.223069, 42.603353),
    'drc-mai-ndombe' : (-3.696119, 20.362077),
    'malawi-salima': (-13.6, 34.32),
    'brazil-para': (-2.064534, -56.578095),
    'brazil-para-2': (-7.351687, -48.457507),
    'pakistan-mardan': (34.355452, 71.945095),
    'botswana-kweneng': (-24.360968, 25.176526),
    'nicaragua-bonanza': (13.933745, -84.690842),
    'ghana-cocoa': (7.398111, -1.269223),
    'ghana-brong-ahafo': (7.70258, -0.70911),
    'mexico-change-det': (21.212083, -88.993677),
    'costa-rica-change-det': (8.47520, -82.94909),
    'honduras-colon': (15.617889, -85.447611),
    'mexico-campeche-change': (18.151747, -92.152278),
    'guatemala-gain': (16.464444, -89.479170),
    'guatemala-gain-2': (15.196480, -89.118290)
}

landscape = 'guatemala-gain-2'

OUTPUT_FOLDER = '../tile_data/{}/'.format(landscape)
coords = landscapes[landscape]
coords = (coords[1], coords[0])
print(OUTPUT_FOLDER, coords)

../tile_data/guatemala-gain-2/ (-89.11829, 15.19648)


In [6]:
landscape_df = pd.DataFrame({'landscape': [x for x in landscapes.keys()], 
                             'latitude': [x[0] for x in landscapes.values()],
                             'longitude': [x[1] for x in landscapes.values()]
})

landscape_df.to_csv("../data/latlongs/landscapes.csv", index=False)

# Projection functions

In [7]:
def calculate_bbx_pyproj(coord: Tuple[float, float],
                         step_x: int, step_y: int,
                         expansion: int, multiplier: int = 1.) -> (Tuple[float, float], 'CRS'):
    ''' Calculates the four corners of a bounding box
        [bottom left, top right] as well as the UTM EPSG using Pyproj
        
        Note: The input for this function is (x, y), not (lat, long)
        
        Parameters:
         coord (tuple): Initial (long, lat) coord
         step_x (int): X tile number of a 6300x6300 meter tile
         step_y (int): Y tile number of a 6300x6300 meter tile
         expansion (int): Typically 10 meters - the size of the border for the predictions
         multiplier (int): Currently deprecated
         
        Returns:
         coords (tuple):
         CRS (int):
    '''
    
    inproj = Proj('epsg:4326')
    outproj_code = calculate_epsg(coord)
    outproj = Proj('epsg:' + str(outproj_code))
    
    
    
    coord_utm =  transform(inproj, outproj, coord[1], coord[0])
    coord_utm_bottom_left = (coord_utm[0] + step_x*6300 - expansion,
                             coord_utm[1] + step_y*6300 - expansion)
    
    coord_utm_top_right = (coord_utm[0] + (step_x+multiplier) * 6300 + expansion,
                           coord_utm[1] + (step_y+multiplier) * 6300 + expansion)

    zone = str(outproj_code)[3:]
    direction = 'N' if coord[1] >= 0 else 'S'
    utm_epsg = "UTM_" + zone + direction
    return (coord_utm_bottom_left, coord_utm_top_right), CRS[utm_epsg]


def pts_in_geojson(lats: List[float], longs: List[float], geojson: 'geojson') -> bool:  
    """ Identifies whether candidate download tile is within an input geojson
        
        Parameters:
         lats (list): list of latitudes
         longs (list): list of longitudes
         geojson (float): path to input geojson
    
        Returns:
         bool 
    """
    polys = geopandas.read_file(geojson)['geometry']
    polys = geopandas.GeoSeries(polys)
    pnts = [Point(x, y) for x, y in zip(list(lats), list(longs))]
    
    def _contains(pt):
        return polys.contains(pt)[0]

    if any([_contains(pt) for pt in pnts]):
        return True
    else: return False

# Data download functions

In [8]:
def identify_clouds(bbox: List[Tuple[float, float]],
                        epsg: 'CRS', dates: dict = dates) -> (np.ndarray, np.ndarray, np.ndarray):

    """ Downloads and calculates cloud cover and shadow
        
        Parameters:
         bbox (list): output of calc_bbox
         epsg (float): EPSG associated with bbox 
         dates (tuple): YY-MM-DD - YY-MM-DD bounds for downloading 
    
        Returns:
         cloud_img (np.array):
         shadows (np.array): 
         clean_steps (np.array):
    """
    box = BBox(bbox, crs = epsg)
    cloud_request = WcsRequest(
        layer='CLOUD_NEW',
        bbox=box,
        time=dates,
        resx='160m', 
        resy='160m',
        image_format = MimeType.TIFF_d8,
        maxcc=0.7,
        instance_id=API_KEY,
        custom_url_params = {constants.CustomUrlParam.UPSAMPLING: 'NEAREST'},
        time_difference=datetime.timedelta(hours=72),
    )


    shadow_request = WcsRequest(
        layer='SHADOW',
        bbox=box,
        time=dates,
        resx='10m',
        resy='10m',
        image_format =  MimeType.TIFF_d16,
        maxcc=0.7,
        instance_id=API_KEY,
        custom_url_params = {constants.CustomUrlParam.UPSAMPLING: 'NEAREST'},
        time_difference=datetime.timedelta(hours=72))

    cloud_img = cloud_request.get_data()
    cloud_img = np.array(cloud_img)
    print(f"The max clouds is {np.max(cloud_img)}")
    print(cloud_img.shape)
    if np.max(cloud_img > 10):
        cloud_img = cloud_img / 255
        
    assert np.max(cloud_img) <= 1.
    c_probs_pus = ((40*40)/(512*512)) *(1/3)*cloud_img.shape[0]
    print(f"Cloud_probs used {c_probs_pus} processing units")
    
    cloud_img = resize(cloud_img, (cloud_img.shape[0], IMSIZE, IMSIZE), order = 0)
    n_cloud_px = np.array([len(np.argwhere(cloud_img[x, :, :].reshape((IMSIZE)*(IMSIZE)) > 0.33))
                           for x in range(cloud_img.shape[0])])
    cloud_steps = np.argwhere(n_cloud_px > IMSIZE**2 / 5)
    clean_steps = [x for x in range(cloud_img.shape[0]) if x not in cloud_steps]
    print(f"Removing {len(cloud_steps)} from S2 download, saving {7.32 * len(cloud_steps)} PU")

    shadow_img = shadow_request.get_data(data_filter = clean_steps)
    shadow_img = np.array(shadow_img)
    print(shadow_img.shape)
    shadow_img = resize(shadow_img, (shadow_img.shape[0], IMSIZE, IMSIZE, shadow_img.shape[-1]), order = 0)
    print(shadow_img.shape)
    shadow_pus = ((IMSIZE*IMSIZE)/(512*512)) * shadow_img.shape[0]
    print(f"Shadows used: {shadow_pus} processing units")
    print(f"The max shadows is {np.max(shadow_img)}")
    if np.max(shadow_img > 10):
        shadow_img = shadow_img / 65535
    print(np.max(shadow_img))
 
    cloud_img = np.delete(cloud_img, cloud_steps, 0)
    shadows = mcm_shadow_mask(np.array(shadow_img), cloud_img)

    print(f"Cloud probs: {cloud_img.shape}")
    print(f"Shadow shape {shadows.shape}")
    return cloud_img, shadows, clean_steps

    
    
def download_dem(bbox: List[Tuple[float, float]], epsg: 'CRS') -> np.ndarray:
    """ Downloads the DEM layer from Sentinel hub
        
        Parameters:
         bbox (list): output of calc_bbox
         epsg (float): EPSG associated with bbox 
    
        Returns:
         dem_image (arr):
    """

    box = BBox(bbox, crs = epsg)
    dem_s = (630)+4+8+8
    dem_request = WmsRequest(data_source=DataSource.DEM,
                         layer='DEM',
                         bbox=box,
                         width=dem_s,
                         height=dem_s,
                         instance_id=API_KEY,
                         image_format=MimeType.TIFF_d32f,
                         custom_url_params={CustomUrlParam.SHOWLOGO: False})
    dem_image = dem_request.get_data()[0]
    dem_image = calcSlope(dem_image.reshape((1, dem_s, dem_s)),
                  np.full((dem_s, dem_s), 10), np.full((dem_s, dem_s), 10), zScale = 1, minSlope = 0.02)
    dem_image = dem_image.reshape((dem_s,dem_s, 1))
    dem_image = dem_image[1:dem_s-1, 1:dem_s-1, :]
    print(f"DEM used {((IMSIZE*IMSIZE)/(512*512))*2} processing units")
    return dem_image
 

def download_layer(bbox: List[Tuple[float, float]],
                   clean_steps: np.ndarray, epsg: 'CRS',
                   dates: dict = dates, year: int = year) -> (np.ndarray, np.ndarray):
    """ Downloads the L2A sentinel layer with 10 and 20 meter bands
        
        Parameters:
         bbox (list): output of calc_bbox
         clean_steps (list): list of steps to filter download request
         epsg (float): EPSG associated with bbox 
         time (tuple): YY-MM-DD - YY-MM-DD bounds for downloading 
    
        Returns:
         img (arr):
         img_request (obj): 
    """
    box = BBox(bbox, crs = epsg)
    image_request = WcsRequest(
            layer='L2A20',
            bbox=box,
            time=dates,
            image_format = MimeType.TIFF_d16,
            maxcc=0.7,
            resx='20m', resy='20m',
            instance_id=API_KEY,
            custom_url_params = {constants.CustomUrlParam.DOWNSAMPLING: 'NEAREST',
                                constants.CustomUrlParam.UPSAMPLING: 'NEAREST'},
            time_difference=datetime.timedelta(hours=72),
        )
    print("Downloading L2A 20m layer")
    img_bands = image_request.get_data(data_filter = clean_steps)
    img_20 = np.stack(img_bands)
    print(f"The max 20m is {np.max(img_20)}")
    if np.max(img_20) >= 10:
        img_20 = img_20 / 65535
    assert np.max(img_20) <= 2.
    
    s2_20_usage = (img_20.shape[1]*img_20.shape[2])/(512*512) * (6/3) * img_20.shape[0]
    print("Original 20 meter bands size: {}, using {} PU".format(img_20.shape, s2_20_usage))
    img_20 = resize(img_20, (img_20.shape[0], IMSIZE, IMSIZE, img_20.shape[-1]), order = 0)

    image_request = WcsRequest(
            layer='L2A10',
            bbox=box,
            time=dates,
            image_format = MimeType.TIFF_d16,
            maxcc=0.7,
            resx='10m', resy='10m',
            instance_id=API_KEY,
            custom_url_params = {constants.CustomUrlParam.DOWNSAMPLING: 'BICUBIC',
                                constants.CustomUrlParam.UPSAMPLING: 'BICUBIC'},
            time_difference=datetime.timedelta(hours=72),
    )
    
    print("Downloading L2A 10m layer")
    img_bands = image_request.get_data(data_filter = clean_steps)
    img_10 = np.stack(img_bands)
    print(f"The max 10m is {np.max(img_10)}")
    if np.max(img_10) >= 10:
        img_10 = img_10 / 65535
    assert np.max(img_10) <= 2.
    
    s2_10_usage = (img_10.shape[1]*img_10.shape[2])/(512*512) * (4/3) * img_10.shape[0]
    print("Original 20 meter bands size: {}, using {} PU".format(img_10.shape, s2_10_usage))
    img_10 = resize(img_10, (img_10.shape[0], IMSIZE, IMSIZE, img_10.shape[-1]), order = 0)
    img = np.concatenate([img_10, img_20], axis = -1)
    print(f"Sentinel 2 used {s2_20_usage + s2_10_usage} PU")

    image_dates = []
    for date in image_request.get_dates():
        if date.year == year - 1:
            image_dates.append(-365 + starting_days[(date.month-1)] + date.day)
        if date.year == year:
            image_dates.append(starting_days[(date.month-1)] + date.day)
        if date.year == year + 1:
            image_dates.append(365 + starting_days[(date.month-1)]+date.day)
    image_dates = [val for idx, val in enumerate(image_dates) if idx in clean_steps]
    image_dates = np.array(image_dates)
    return img, image_dates

        
def download_sentinel_1(bbox: List[Tuple[float, float]],
                        epsg: 'CRS', imsize: int = IMSIZE, 
                        dates: dict = dates_sentinel_1, layer: str = "SENT",
                        year: int = year) -> (np.ndarray, np.ndarray):
    """ Downloads the GRD Sentinel 1 VV-VH layer from Sentinel Hub
        
        Parameters:
         bbox (list): output of calc_bbox
         epsg (float): EPSG associated with bbox 
         imsize (int):
         dates (tuple): YY-MM-DD - YY-MM-DD bounds for downloading 
         layer (str):
         year (int): 
    
        Returns:
         s1 (arr):
         image_dates (arr): 
    """
    box = BBox(bbox, crs = epsg)
    image_request = WcsRequest(
            layer=layer,
            bbox=box,
            time=dates,
            image_format = MimeType.TIFF_d16,
            data_source=DataSource.SENTINEL1_IW,
            maxcc=1.0,
            resx='10m', resy='5m',
            instance_id=API_KEY,
            custom_url_params = {constants.CustomUrlParam.DOWNSAMPLING: 'NEAREST',
                                constants.CustomUrlParam.UPSAMPLING: 'NEAREST'},
            time_difference=datetime.timedelta(hours=96),
        )
    data_filter = None
    if len(image_request.download_list) > 50:
        data_filter = [x for x in range(len(image_request.download_list)) if x % 2 == 0]
    img_bands = image_request.get_data(data_filter = data_filter)
    s1 = np.stack(img_bands)
    if np.max(s1) >= 1000:
            s1 = s1 / 65535.
    
    print(f"The max s1 is {np.max(s1)}")
    print(f"Sentinel 1 used {(2/3)*s1.shape[0] * (s1.shape[1]*s1.shape[2])/(512*512)} PU for \
          {s1.shape[0]} out of {len(image_request.download_list)} images")
    s1 = resize(s1, (s1.shape[0], imsize*2, imsize*2, s1.shape[-1]), order = 0)
    s1 = np.reshape(s1, (s1.shape[0], s1.shape[1]//2, 2, s1.shape[2] // 2, 2, s1.shape[-1]))
    s1 = np.mean(s1, (2, 4))

    image_dates = []
    for date in image_request.get_dates():
        if date.year == year - 1:
            image_dates.append(-365 + starting_days[(date.month-1)] + date.day)
        if date.year == year:
            image_dates.append(starting_days[(date.month-1)] + date.day)
        if date.year == year + 1:
            image_dates.append(365 + starting_days[(date.month-1)]+date.day)
    image_dates = np.array(image_dates)
    s1c = np.copy(s1)
    s1c[np.where(s1c < 1.)] = 0
    n_pix_oob = np.sum(s1c, axis = (1, 2, 3))
    to_remove = np.argwhere(n_pix_oob > (imsize*2*imsize*2)/50)
    s1 = np.delete(s1, to_remove, 0)
    image_dates = np.delete(image_dates, to_remove)
    return s1, image_dates


def identify_s1_layer(coords: Tuple[float, float]) -> str:
    """ Identifies whether to download ascending or descending 
        sentinel 1 orbit based upon predetermined geographic coverage
        
        Reference: https://sentinel.esa.int/web/sentinel/missions/
                   sentinel-1/satellite-description/geographical-coverage
        
        Parameters:
         coords (tuple): 
    
        Returns:
         layer (str): either of SENT, SENT_DESC 
    """
    results = rg.search(coords)
    country = results[-1]['cc']
    continent_name = pc.country_alpha2_to_continent_code(country)
    if continent_name in ['AF', 'OC']:
        layer = "SENT"
    if continent_name in ['SA']:
        if coords[0] > -7.11:
            layer = "SENT"
        else:
            layer = "SENT_DESC"
    if continent_name in ['AS']:
        if coords[0] > 23.3:
            layer = "SENT"
        else:
            layer = "SENT_DESC"
    if continent_name in ['NA']:
        layer = "SENT_DESC"
    print(continent_name)
    print(layer)
    return layer

# Cloud and shadow removal

In [9]:
def remove_missed_clouds(img: np.ndarray) -> np.ndarray:
    """ Removes clouds that may have been missed by s2cloudless
        by looking at a temporal change outside of IQR
        
        Parameters:
         img (arr): 
    
        Returns:
         to_remove (arr): 
    """
    
    iqr = np.percentile(img[:, :, :, 3].flatten(), 75) - np.percentile(img[:, :, :, 3].flatten(), 25)
    thresh_t = np.percentile(img[:, :, :, 3].flatten(), 75) + iqr*2
    thresh_b = np.percentile(img[:, :, :, 3].flatten(), 25) - iqr*2
    diffs_fw = np.diff(img, 1, axis = 0)
    diffs_fw = np.mean(diffs_fw, axis = (1, 2, 3))
    diffs_fw = np.array([0] + list(diffs_fw))
    diffs_bw = np.diff(np.flip(img, 0), 1, axis = 0)
    diffs_bw = np.flip(np.mean(diffs_bw, axis = (1, 2, 3)))
    diffs_bw = np.array(list(diffs_bw) + [0])
    diffs = abs(diffs_fw - diffs_bw) * 100
    outlier_percs = []
    for step in range(img.shape[0]):
        bottom = len(np.argwhere(img[step, :, :, 3].flatten() > thresh_t))
        top = len(np.argwhere(img[step, :, :, 3].flatten() < thresh_b))
        p = 100* ((bottom + top) / (IMSIZE*IMSIZE))
        outlier_percs.append(p)
    to_remove = np.argwhere(np.array(outlier_percs) > 20)
    return to_remove

def calculate_bad_steps(sentinel2: np.ndarray, clouds: np.ndarray) -> np.ndarray:
    """ Calculates the timesteps to remove based upon cloud cover and missing data
        
        Parameters:
         sentinel2 (arr): 
         clouds (arr):
    
        Returns:
         to_remove (arr): 
    """
    n_cloud_px = np.array([len(np.argwhere(clouds[x, :, :].reshape((IMSIZE)*(IMSIZE)) > 0.30)) for x in range(clouds.shape[0])])
    cloud_steps = np.argwhere(n_cloud_px > IMSIZE**2 / 7)
    print(f'The percent cloud cover is {n_cloud_px/(IMSIZE**2)}')
    missing_images = [np.argwhere(sentinel2[x, :, : :10].flatten() == 0.0) for x in range(sentinel2.shape[0])]
    missing_images = np.array([len(x) for x in missing_images])
    print(f'The number of missing 0 is {missing_images/(IMSIZE**2)}')
    missing_images_p = [np.argwhere(sentinel2[x, :, : :10].flatten() >= 1) for x in range(sentinel2.shape[0])]
    missing_images_p = np.array([len(x) for x in missing_images_p])
    print(f'The number of missing 1 is {missing_images_p/(IMSIZE**2)}')
    missing_images += missing_images_p
    missing_images = np.argwhere(missing_images >= (IMSIZE**2) / 20)
    to_remove = np.unique(np.concatenate([cloud_steps.flatten(), missing_images.flatten()]))
    return to_remove

# Superresolution

In [10]:
MDL_PATH = "../src/dsen2/models/"

input_shape = ((4, None, None), (6, None, None))
model = s2model(input_shape, num_layers=6, feature_size=128)
predict_file = MDL_PATH+'s2_032_lr_1e-04.hdf5'
print('Symbolic Model Created.')

model.load_weights(predict_file)

def DSen2(d10: np.ndarray, d20: np.ndarray) -> np.ndarray:
    """Super resolves 20 meter bans using the DSen2 convolutional
       neural network, as specified in Lanaras et al. 2018
       https://github.com/lanha/DSen2

        Parameters:
         d10 (arr): (4, X, Y) shape array with 10 meter resolution
         d20 (arr): (6, X, Y) shape array with 20 meter resolution

        Returns:
         prediction (arr): (6, X, Y) shape array with 10 meter superresolved
                          output of DSen2 on d20 array
    """
    test = [d10, d20]
    input_shape = ((4, None, None), (6, None, None))
    prediction = _predict(test, input_shape, deep=False)
    return prediction

def _predict(test: np.ndarray, input_shape: Tuple, model: 'model' = model,
             deep: bool = False, run_60: bool = False) -> np.ndarray:
    
    prediction = model.predict(test, verbose=0, batch_size = 8)
    return prediction

def superresolve(sentinel2: np.ndarray) -> np.ndarray:
    """Worker function to deal with types and shapes
       to superresolve a 10-band input array

        Parameters:
         sentinel2 (arr): (:, X, Y, 10) shape array with 10 meter resolution
                          bands in indexes 0-4, and 20 meter in 4- 10

        Returns:
         superresolved (arr): (:, X, Y, 10) shape array with 10 meter 
                              superresolved output of DSen2
    """
    d10 = sentinel2[:, :, :, 0:4]
    d20 = sentinel2[:, :, :, 4:10]

    d10 = np.swapaxes(d10, 1, -1)
    d10 = np.swapaxes(d10, 2, 3)
    d20 = np.swapaxes(d20, 1, -1)
    d20 = np.swapaxes(d20, 2, 3)
    superresolved = DSen2(d10, d20)
    superresolved = np.swapaxes(superresolved, 1, -1)
    superresolved = np.swapaxes(superresolved, 1, 2)
    sentinel2[:, :, :, 4:10] = superresolved
    return sentinel2 # returns band IDXs 3, 4, 5, 7, 8, 9

def superresolve_tile(arr):
    print(f"The input array to superresolve is {arr.shape}")
    superresolved = np.copy(arr)
    tiles = tile_window(646, 646, 56, 56)
    for i in tnrange(len(tiles)):
        subtile = tiles[i]
        to_resolve = arr[:, subtile[0]:subtile[0]+56, subtile[1]:subtile[1]+56, :]
        to_resolve = np.pad(to_resolve, ((0, 0), (4, 4), (4, 4), (0, 0)), 'reflect')
        resolved = superresolve(to_resolve)
        resolved = resolved[:, 4:-4, 4:-4, :]
        superresolved[:, subtile[0]:subtile[0]+56, subtile[1]:subtile[1]+56] = resolved
    return superresolved

Instructions for updating:
Colocations handled automatically by placer.
Symbolic Model Created.


# Tiling and folder management functions

In [20]:
def make_output_and_temp_folders(idx, output_folder = OUTPUT_FOLDER):
    
    def _find_and_make_dirs(dirs):
        if not os.path.exists(os.path.realpath(dirs)):
            os.makedirs(os.path.realpath(dirs))
            
    _find_and_make_dirs(output_folder + "raw/")
    _find_and_make_dirs(output_folder + "raw/clouds/")
    _find_and_make_dirs(output_folder + "raw/s1/")
    _find_and_make_dirs(output_folder + "raw/s2/")
    _find_and_make_dirs(output_folder + "raw/misc/")
    _find_and_make_dirs(output_folder + "processed/")
    

def check_contains(coord, step_x, step_y, folder = OUTPUT_FOLDER):
    contains = False
    bbx, epsg = calculate_bbx_pyproj(coord, step_x, step_y, expansion = 80)
    inproj = Proj('epsg:' + str(str(epsg)[5:]))
    outproj = Proj('epsg:4326')
    bottomleft = transform(inproj, outproj, bbx[0][0], bbx[0][1])
    topright = transform(inproj, outproj, bbx[1][0], bbx[1][1])
    
    if os.path.exists(folder):
            if any([x.endswith(".geojson") for x in os.listdir(folder)]):
                geojson_path = folder + [x for x in os.listdir(folder) if x.endswith(".geojson")][0]
    
                bool_contains = pts_in_geojson(lats = [bottomleft[1], topright[1]], 
                                                       longs = [bottomleft[0], topright[0]],
                                                       geojson = geojson_path)
                contains = bool_contains
    return contains

def download_large_tile(coord, step_x, step_y, folder = OUTPUT_FOLDER, year = year, s1_layer = "SENT"):
    
    bbx, epsg = calculate_bbx_pyproj(coord, step_x, step_y, expansion = 80)
    dem_bbx, _ = calculate_bbx_pyproj(coord, step_x, step_y, expansion = 90)
    idx = str(step_y) + "_" + str(step_x)
    idx = str(idx)
    make_output_and_temp_folders(idx)

    print("Calculating cloud cover")
    if not os.path.exists(folder + "output/" + str(((step_y+1)*5)-5) + "/" + str(((step_x+1)*5)-5) + ".npy"):
        if not os.path.exists(folder + "processed/" + str(((step_y+1)*5)-5) + "/" + str(((step_x+1)*5)-5) + ".hkl"):
            if not os.path.exists(folder + "raw/clouds/clouds_{}.hkl".format(idx)):
                cloud_probs, shadows, clean_steps = identify_clouds(bbx, epsg = epsg)
                hkl.dump(cloud_probs, folder + "raw/clouds/clouds_{}.hkl".format(idx), mode='w', compression='gzip')
                hkl.dump(shadows, folder + "raw/clouds/shadows_{}.hkl".format(idx), mode='w', compression='gzip')
                hkl.dump(clean_steps, folder + "raw/clouds/clean_steps_{}.hkl".format(idx), mode='w', compression='gzip')
            
            if not os.path.exists(folder + "raw/s1/{}.hkl".format(idx)):
                print("Downloading S1")
                s1_layer = identify_s1_layer((coord[1], coord[0]))
                s1, s1_dates = download_sentinel_1(bbx, layer = s1_layer, epsg = epsg)
                if s1.shape[0] == 0:
                    s1_layer = "SENT_DESC" if s1_layer == "SENT" else "SENT"
                    print(f'Switching to {s1_layer}')
                    s1, s1_dates = download_sentinel_1(bbx, layer = s1_layer, epsg = epsg)
                s1 = process_sentinel_1_tile(s1, s1_dates)
                hkl.dump(s1, folder + "raw/s1/{}.hkl".format(idx), mode='w', compression='gzip')
                hkl.dump(s1_dates, folder + "raw/misc/s1_dates_{}.hkl".format(idx), mode='w', compression='gzip')

            if not os.path.exists(folder + "raw/s2/{}.hkl".format(idx)):
                print("Downloading S2")
                if 'clean_steps' not in globals() or locals():
                    clean_steps = hkl.load(folder + "raw/clouds/clean_steps_{}.hkl".format(idx))
                s2, s2_dates = download_layer(bbx, clean_steps = clean_steps, epsg = epsg)
                hkl.dump(s2, folder + "raw/s2/{}.hkl".format(idx), mode='w', compression='gzip')
                hkl.dump(s2_dates, folder + "raw/misc/s2_dates_{}.hkl".format(idx), mode='w', compression='gzip')

            if not os.path.exists(folder + "raw/misc/dem_{}.hkl".format(idx)):
                print("Downloading DEM")
                dem = download_dem(dem_bbx, epsg = epsg) # get the DEM BBOX
                hkl.dump(dem, folder + "raw/misc/dem_{}.hkl".format(idx), mode='w', compression='gzip')

In [21]:
def process_sentinel_1_tile(sentinel1, dates):
    s1, _ = calculate_and_save_best_images(sentinel1, dates)
    biweekly_dates = np.array([day for day in range(0, 360, 5)])
    to_remove = np.argwhere(biweekly_dates % 15 != 0)
    s1 = np.delete(s1, to_remove, 0)
    return s1

def convert_to_int16(array):
    return np.trunc(array * 65535).astype(int)

def make_folder_names(step_x, step_y):
    x_vals = []
    y_vals = []
    for i in range(25):
        y_val = (24 - i) // 5
        x_val = 5 - ((25 - i) % 5)
        x_val = 0 if x_val == 5 else x_val
        x_vals.append(x_val)
        y_vals.append(y_val)
    y_vals = [i + (5*step_y) for i in y_vals]
    x_vals = [i + (5*step_x) for i in x_vals]
    return x_vals, y_vals


def process_large_tile(coord, step_x, step_y, folder = OUTPUT_FOLDER):
    idx = str(step_y) + "_" + str(step_x)
    x_vals, y_vals = make_folder_names(step_x, step_y)

    processed = True
    for x, y in zip(x_vals, y_vals):
        folder_path = f"{str(y)}/{str(x)}"
        processed_exists = os.path.exists(folder + "processed/" + folder_path + ".hkl")
        output_exists = os.path.exists(folder + "output/" + folder_path + ".npy")
        if not (processed_exists or output_exists):
            processed = False
    if not processed:
        clouds = hkl.load(folder + "raw/clouds/clouds_{}.hkl".format(idx))
        sentinel1 = hkl.load(folder + "raw/s1/{}.hkl".format(idx))
        radar_dates = hkl.load(folder + "raw/misc/s1_dates_{}.hkl".format(idx))
        sentinel2 = hkl.load(folder + "raw/s2/{}.hkl".format(idx))
        dem = hkl.load(folder + "raw/misc/dem_{}.hkl".format(idx))
        image_dates = hkl.load(folder + "raw/misc/s2_dates_{}.hkl".format(idx))
        if os.path.exists(folder + "raw/clouds/shadows_{}.hkl".format(idx)):
            shadows = hkl.load(folder + "raw/clouds/shadows_{}.hkl".format(idx))
        else:
            print("No shadows file, so calculating shadows with L2A")
            shadows = mcm_shadow_mask(sentinel2, clouds)
        print("The files have been loaded")

        to_remove = calculate_bad_steps(sentinel2, clouds)
        sentinel2 = np.delete(sentinel2, to_remove, axis = 0)
        clouds = np.delete(clouds, to_remove, axis = 0)
        shadows = np.delete(shadows, to_remove, axis = 0)
        image_dates = np.delete(image_dates, to_remove)
        print(f"{len(to_remove)} Cloudy and missing images removed, radar processed")
        to_remove = remove_missed_clouds(sentinel2)
        print(f"{len(to_remove)} missed cloudy images should have been removed")
        x = remove_cloud_and_shadows(sentinel2, clouds, shadows, image_dates)
        print("Clouds and shadows interpolated")       
        
        index = 0
        print("Super resolving tile")
        x = superresolve_tile(x)
        print(f"The superresolved shape is: {x.shape}")
        
        tiles = tile_window(IMSIZE, IMSIZE, window_size = 142)
        for t in tiles:
            print(index)
            index += 1
            start_x = t[0]
            start_y = t[1]
            end_x = start_x + t[2]
            end_y = start_y + t[3]
            print(index)
            output_file = f"{folder}processed/{y_vals[index]}/{x_vals[index]}.hkl"
            if not os.path.exists(output_file):
            #if not os.path.exists(folder + "processed/{}/{}.hkl".format(str(y_vals[index]), str(x_vals[index]))):
                subtile = x[:, start_x:end_x, start_y:end_y, :]
                dem_i = np.tile(dem[np.newaxis, start_x:end_x, start_y:end_y, :], (x.shape[0], 1, 1, 1))
                subtile = np.concatenate([subtile, dem_i / 90], axis = -1)
                subtile = evi(subtile, verbose = True)
                subtile = bi(subtile, verbose = True)
                subtile = msavi2(subtile, verbose = True)
                subtile = si(subtile, verbose = True)
                t3 = timer()

                subtile, _ = calculate_and_save_best_images(subtile, image_dates)
                subtile = interpolate_array(subtile, dim = 142)
                t5 = timer()
                print("Interpolate: {}".format(t5 - t3))
                subtile = np.concatenate([subtile, sentinel1[:, start_x:end_x,
                                                            start_y:end_y, :]], axis = -1)

                out_y_folder = folder + "processed/{}/".format(str(y_vals[index]))
                if not os.path.exists(os.path.realpath(out_y_folder)):
                    os.makedirs(os.path.realpath(out_y_folder))
                subtile = convert_to_int16(subtile)
                assert subtile.shape[1] == 142, f"subtile shape is {subtile.shape}"
                hkl.dump(subtile, output_file,
                         #folder + "processed/{}/{}.hkl".format(str(y_vals[index]), str(x_vals[index])),
                         mode='w', compression='gzip')
            
def clean_up_folders():
    pass

# Function execution

In [22]:
expansion = -10
multiplier = 1
step_x = 1
coords_init = offset_x(coords, 0)
coords_init[1] += 0

coord1 = offset_x(coords, 6300*(step_x + multiplier) + expansion)
coord1[1] += 6300*(step_x + multiplier) + expansion


calculate_area([coords_init, coord1])

-89.11829 15.19648
-89.11829 15.19648
15850


In [ ]:
downloaded = 0

if not os.path.exists(os.path.realpath(OUTPUT_FOLDER)):
            os.makedirs(os.path.realpath(OUTPUT_FOLDER))
        
for x_tile in range(0, 1):
    for y_tile in range(0, 2):
        contains = True
        #contains = check_contains(coords, x_tile, y_tile, OUTPUT_FOLDER)
        print(contains)
        if contains:
            print("X: {} Y:{}".format(x_tile, y_tile))
            downloaded += 1
            print(f"Downloaded {downloaded}")
            download_large_tile(coord = coords, step_x = x_tile, step_y = y_tile)
            process_large_tile(coords, x_tile, y_tile)
            print("\n")
            #clean_up_folders(x_tile, y_tile)

True
X: 0 Y:0
Downloaded 1
-89.11829 15.19648
-89.11829 15.19648
Calculating cloud cover


True
X: 0 Y:1
Downloaded 2
-89.11829 15.19648
-89.11829 15.19648
Calculating cloud cover
The max clouds is 255
(68, 40, 40)
Cloud_probs used 0.13834635416666666 processing units
Removing 43 from S2 download, saving 314.76 PU
(25, 646, 646, 3)
(25, 646, 646, 3)
Shadows used: 39.79835510253906 processing units
The max shadows is 1.0
1.0
648
646



Cloud probs: (25, 646, 646)
Shadow shape (25, 646, 646)
Loading formatted geocoded file...
NA
SENT_DESC
The max s1 is 1.0
Sentinel 1 used 63.6773681640625 PU for           30 out of 60 images
Maximum time distance: 0
The max 20m is 65535
Original 20 meter bands size: (25, 323, 323, 6), using 19.89917755126953 PU
The max 10m is 65535
Original 20 meter bands size: (25, 646, 646, 4), using 53.06447347005208 PU
Sentinel 2 used 72.96365102132161 PU
DEM used 3.183868408203125 processing units
The files have been loaded
The percent cloud cover is [0.10394521 0.         0.12979133 0.06161278 0.17169723 0.00184033
 0.12488138 0.00184033 0.11395681 0.08780397 0.02035867 0.05356373
 0.         0.00187867 0.19738759 0.01303569 0.00122689 0.00306722
 0.15690268 0.19601932 0.00184033 0.00061344 0.0558234  0.11840907
 0.00617278]
The number of missing 0 is [0.         0.         0.         0.         0.         0.18566266
 0.         0.         0.         0.         0.         0.
 0.         0.     